In [1]:
import pandas as pd
import numpy as np
import pycmap
from datetime import datetime, timedelta
import pytz
import matplotlib.pyplot as plt
import os
import glob
import plotly.express as px
from astral import Observer
from astral.sun import sun
from astral.sun import sunrise
from astral.sun import sunset
from astral.sun import zenith

In [2]:
df = pd.read_csv('G4_par_wind.csv')
df['time'] = pd.DatetimeIndex(df.time.values)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [3]:
sat_par = pd.read_csv('/Users/ceg/Desktop/kristine/SACNAS-2023-project/sat_par.csv')
sat_par['Date'] = pd.DatetimeIndex(sat_par.Date.values)
sat_par.drop(['Unnamed: 0', 'cruise', 'PAR'], axis = 1, inplace = True)

In [4]:
df1 = pd.read_csv('Gradients4_TN397_Optics_LISST_ACS_ECO.csv')
df1['time'] = pd.DatetimeIndex(df1.time.values)
df2 = df1.groupby([pd.Grouper(key = 'time', freq = 'H')]).agg({'chla_eco': 'mean'}).reset_index()
g4_chloro = df2[df2['chla_eco'].notnull()]

In [5]:
chla_satpar = pd.merge(g4_chloro, sat_par, left_on = 'time', right_on = 'Date', how = 'inner')
chla_satpar.drop(['Date'], axis = 1, inplace = True)

In [6]:
g4_data = pd.merge(df, chla_satpar, left_on = 'time', right_on = 'time', how = 'inner')
g4_data

## rename par and sat_par column as calibrated and uncalibrated par

,time,licor_par,cruise,pop,n_per_uL,c_per_uL,lat,lon,temp,salinity,par,diam_med,hour,round_lat,round_lon,scalar,chla_eco,sat_par
0,2021-11-19 17:00:00+00:00,486.087167,TN397_740,prochloro,59.103647,1.471909,31.047502,-119.963323,17.614825,33.200625,909.702550,0.024904,17,31.0,-120.0,5.205841,0.234573,576.539448
1,2021-11-19 18:00:00+00:00,891.885167,TN397_740,prochloro,73.904420,1.901776,30.962975,-120.109363,17.633781,33.158950,1619.012500,0.025733,18,31.0,-120.0,5.037268,0.163371,1088.754558
2,2021-11-19 19:00:00+00:00,1092.274333,TN397_740,prochloro,60.774550,1.420525,30.799496,-120.382978,18.038000,33.079418,1876.892611,0.023374,19,31.0,-120.5,5.219913,0.107802,1286.427594
3,2021-11-19 20:00:00+00:00,1002.763667,TN397_740,prochloro,55.664875,1.528290,30.685723,-120.577657,17.635111,33.002106,1953.495900,0.027455,20,30.5,-120.5,5.233759,0.146960,1217.701810
4,2021-11-19 21:00:00+00:00,922.692167,TN397_740,prochloro,63.412060,1.730710,30.605379,-120.714316,17.676354,33.059641,2278.979714,0.027293,21,30.5,-120.5,5.602349,0.139737,1091.592987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468,2021-12-13 18:00:00+00:00,688.223000,TN397_740,prochloro,241.180751,8.157731,14.772220,-152.864218,26.781531,33.951840,1866.283750,0.033824,18,15.0,-153.0,9.271543,0.077962,1210.487611
469,2021-12-13 19:00:00+00:00,1092.185333,TN397_740,prochloro,245.492471,8.885463,14.777021,-152.867004,26.787131,33.953178,2020.028100,0.036194,19,15.0,-153.0,9.141849,0.068782,1266.557619
470,2021-12-13 20:00:00+00:00,1320.478000,TN397_740,prochloro,244.086001,9.166140,14.812232,-152.894542,26.764932,33.947510,2178.220650,0.037553,20,15.0,-153.0,9.028236,0.050990,1365.744348
471,2021-12-13 21:00:00+00:00,1379.093846,TN397_740,prochloro,254.518232,10.394303,14.977258,-153.025971,26.662743,33.941427,2250.400850,0.040839,21,15.0,-153.0,8.841401,0.037757,1409.942455


In [13]:
## making for loop for finding sun zenith angle for each hour
g4_data['sun_zen_angle'] = 'NaN'
for index, row in g4_data.iterrows():
    obs = Observer(row['round_lat'], row['round_lon'], 0)
    zen = zenith(obs, dateandtime = row['time'])
    g4_data.loc[index, 'sun_zen_angle'] = zen
    

In [17]:
g4_data#.to_csv('g4_merged_data')